# Import packages

In [2]:
from loguru import logger
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import VolumeWeightedAveragePrice
import pandas as pd

# Import Data

In [3]:
# Load each CSV file into a DataFrame
df_15m = pd.read_csv('spot_klines_data/BTCUSDT_15m_2024-2025.csv')
df_1d = pd.read_csv('spot_klines_data/BTCUSDT_1d_2024-2025.csv')
df_1m = pd.read_csv('spot_klines_data/BTCUSDT_1m_2024-2025.csv')
df_1w = pd.read_csv('spot_klines_data/BTCUSDT_1w_2024-2025.csv')
df_5m = pd.read_csv('spot_klines_data/BTCUSDT_5m_2024-2025.csv')

# Strategy

In [4]:

class Strategy:
    def __init__(self, data, timeframe_type):
        self.data = data  # DataFrame containing historical price data
        self.timeframe_type = timeframe_type  # 'trend', 'intraday', or 'confirmation'

    def calculate_indicators(self):
        # Based on timeframe_type, select appropriate indicators
        if self.timeframe_type == 'trend':  # Higher Time Frame (HTF - 15M)
            # Trend indicators
            self.data['EMA50'] = EMAIndicator(close=self.data['close'], window=50).ema_indicator()
            self.data['EMA200'] = EMAIndicator(close=self.data['close'], window=200).ema_indicator()
            self.data['RSI'] = RSIIndicator(close=self.data['close'], window=14).rsi()

            # Volume Profile (example: using VWAP as a proxy)
            self.data['VWAP'] = VolumeWeightedAveragePrice(
                high=self.data['high'],
                low=self.data['low'],
                close=self.data['close'],
                volume=self.data['volume'],
                window=20
            ).volume_weighted_average_price()

        elif self.timeframe_type == 'intraday':  # Intermediate Time Frame (ITF - 5M)
            # Momentum and trend continuation indicators
            self.data['RSI'] = RSIIndicator(close=self.data['close'], window=14).rsi()

            macd = MACD(close=self.data['close'], window_slow=26, window_fast=12, window_sign=9)
            self.data['MACD'] = macd.macd()
            self.data['MACD_signal'] = macd.macd_signal()
            self.data['MACD_hist'] = self.data['MACD'] - self.data['MACD_signal']

            # Bollinger Bands for volatility
            bollinger = BollingerBands(close=self.data['close'], window=20, window_dev=2)
            self.data['BB_upper'] = bollinger.bollinger_hband()
            self.data['BB_lower'] = bollinger.bollinger_lband()

        elif self.timeframe_type == 'confirmation':  # Lower Time Frame (LTF - 1M)
            # Precision entry indicators
            self.data['VWAP'] = VolumeWeightedAveragePrice(
                high=self.data['high'],
                low=self.data['low'],
                close=self.data['close'],
                volume=self.data['volume'],
                window=20
            ).volume_weighted_average_price()

            self.data['Stoch_RSI'] = StochasticOscillator(
                high=self.data['high'],
                low=self.data['low'],
                close=self.data['close'],
                window=14,
                smooth_window=3
            ).stoch()

            self.data['ATR'] = AverageTrueRange(
                high=self.data['high'],
                low=self.data['low'],
                close=self.data['close'],
                window=14
            ).average_true_range()

            # Short-term EMAs for dynamic support/resistance
            self.data['EMA9'] = EMAIndicator(close=self.data['close'], window=9).ema_indicator()
            self.data['EMA21'] = EMAIndicator(close=self.data['close'], window=21).ema_indicator()

        else:
            raise ValueError("Invalid timeframe_type. Use 'trend', 'intraday', or 'confirmation'.")

        return self.data

    def preprocess_data(self):
        # Drop NaN values after adding indicators
        self.data.dropna(inplace=True)
        return self.data

    def generate_signals(self):
        self.calculate_indicators()
        self.preprocess_data()

        if self.data.empty:
            raise ValueError("Error: DataFrame is empty before setting signals.")

        self.data['Signal'] = 0  # Default value

        if self.timeframe_type == 'trend':
            # Buy signal: Uptrend confirmed (EMA50 > EMA200, RSI > 50)
            self.data.loc[
                (self.data['EMA50'] > self.data['EMA200']) & 
                (self.data['RSI'] > 50), 'Signal'
            ] = 1

        elif self.timeframe_type == 'intraday':
            # Buy signal: Pullback to support (RSI < 40, MACD histogram > 0)
            self.data.loc[
                (self.data['RSI'] < 40) & 
                (self.data['MACD_hist'] > 0), 'Signal'
            ] = 1

        elif self.timeframe_type == 'confirmation':
            # Buy signal: Price above VWAP, Stoch_RSI oversold (< 20), and ATR confirms volatility
            self.data.loc[
                (self.data['close'] > self.data['VWAP']) & 
                (self.data['Stoch_RSI'] < 20) & 
                (self.data['ATR'] > self.data['ATR'].shift(1)), 'Signal'
            ] = 1

        return self.data

# Fetch and process data

In [5]:
def fetch_and_process_data(data, timeframe, close_time, augmentation=0):
    """
    Fetch and process data for a given timeframe and close_time.

    Parameters:
        data (str or pd.DataFrame): Path to CSV file or a DataFrame containing historical data.
        timeframe (str): Timeframe (e.g., "1m", "5m", "1h", "1d", "1w").
        close_time (str or datetime): The end time for data selection.
        augmentation (int): Number of additional data points to include after 200.

    Returns:
        pd.DataFrame: Filtered dataset containing the last 200 + augmentation points.
    """
    # Load data if it's a file path
    if isinstance(data, str):
        df = pd.read_csv(data)
    elif isinstance(data, pd.DataFrame):
        df = data.copy()
    else:
        raise ValueError("`data` must be a file path (str) or a pandas DataFrame.")

    # Ensure 'close_time' column exists
    if 'close_time' not in df.columns:
        raise KeyError("Missing 'close_time' column in the dataset.")

    # Convert 'close_time' to datetime
    df['close_time'] = pd.to_datetime(df['close_time'])
    close_time = pd.to_datetime(close_time)

    # Define timeframe intervals
    timeframe_intervals = {
        "1m": "T",    # Minute
        "5m": "5T",   # 5 Minutes
        "15m": "15T", # 15 Minutes
        "30m": "30T", # 30 Minutes
        "1h": "H",    # Hour
        "4h": "4H",   # 4 Hours
        "1d": "D",    # Day
        "1w": "W"     # Week
    }

    if timeframe not in timeframe_intervals:
        raise ValueError(f"Unsupported timeframe: {timeframe}")

    # Step 1: Fetch 200 historical data points (before or at `close_time`)
    df_past = df[df['close_time'] <= close_time].sort_values(by='close_time', ascending=False).head(200)
    
    # Step 2: Fetch additional `augmentation` points after `close_time`
    df_future = df[df['close_time'] > close_time].sort_values(by='close_time', ascending=True).head(augmentation)

    # Combine past and future data
    df_final = pd.concat([df_past, df_future]).sort_values(by='close_time', ascending=True).reset_index(drop=True)

    return df_final


# Apply Strategy with a Scheduling function

In [6]:
startegy = Strategy(df_5m, timeframe_type="trend")
df = startegy.generate_signals()
positives = df[df["Signal"] == 1]
positives

,open_time,open,high,low,close,volume,close_time,EMA50,EMA200,RSI,VWAP,Signal
200,2024-01-01 15:40:00,42653.79,42847.07,42636.00,42845.96,176.13327,2024-01-01 15:44:59.999,42661.813496,42540.014394,66.256043,42657.779038,1
201,2024-01-01 15:45:00,42845.95,42846.76,42761.10,42766.01,133.08796,2024-01-01 15:49:59.999,42665.899634,42542.263106,57.722034,42671.949731,1
202,2024-01-01 15:50:00,42766.01,42797.97,42741.20,42787.22,107.81524,2024-01-01 15:54:59.999,42670.657295,42544.700488,59.222596,42680.424093,1
203,2024-01-01 15:55:00,42787.23,42810.00,42756.80,42783.05,88.21628,2024-01-01 15:59:59.999,42675.064852,42547.072125,58.780867,42687.471842,1
204,2024-01-01 16:00:00,42783.05,42805.79,42761.68,42764.00,69.17455,2024-01-01 16:04:59.999,42678.552505,42549.230611,56.700227,42695.866819,1
...,...,...,...,...,...,...,...,...,...,...,...,...
114256,2025-01-31 16:20:00,105176.47,105284.72,105087.76,105283.99,109.36960,2025-01-31 16:24:59.999,104929.574334,104691.265041,54.673457,105238.297954,1
114261,2025-01-31 16:45:00,104988.04,105078.26,104975.43,105078.26,56.86098,2025-01-31 16:49:59.999,104930.552548,104702.980745,50.529819,105292.784981,1
114262,2025-01-31 16:50:00,105078.25,105078.26,104975.00,105062.66,77.75307,2025-01-31 16:54:59.999,104935.733232,104706.559643,50.153275,105307.870520,1
114263,2025-01-31 16:55:00,105062.66,105101.06,105003.11,105081.87,86.73643,2025-01-31 16:59:59.999,104941.464086,104710.294074,50.641052,105340.308691,1


In [7]:
def backtest_multi_timeframe(df_15m, df_5m, df_1m, gat=25, augmentation=15):
    strategy_15m = Strategy(df_15m, 'trend')
    df_15m = strategy_15m.generate_signals()
    trades = []
    for index, row in df_15m.iterrows():
        if row['Signal'] == 1:
            logger.info(f"Condition meet 15m")
            for aug in range(0, gat):
                close_time = row['close_time']
                df_5m_filtered = fetch_and_process_data(df_5m, '5m', close_time, aug)
                strategy_5m = Strategy(df_5m_filtered, 'intraday')
                df_5m_filtered = strategy_5m.generate_signals()
                if df_5m_filtered["Signal"].iloc[-1] == 1:
                    logger.info(f"Condition meet 5m at {aug}")
                    break
                else:
                    logger.info(f"Condition meet until 15m  and no opportunity in 5m for data point {aug}")
                    
            for aug in range(0, augmentation):
                df_1m_filtered = fetch_and_process_data(df_1m, '1m', close_time, aug)
                strategy_1m = Strategy(df_1m_filtered, 'confirmation')
                df_1m_filtered = strategy_1m.generate_signals()                
                if df_1m_filtered['Signal'].iloc[-1] == 1:
                    logger.info(f"Condition meet 1m at {aug}")
                    trades.append({'15m_time': close_time, '5m_confirmed': True, '1m_confirmed': True})
                    logger.info(trades)
                    break
                else:
                    logger.info(f"Condition meet until 5m  and no opportunity in 1m for data point {aug}")

        else:
            logger.info("No opportunity for 15m ")

        


# Example usage
df_15m = pd.read_csv('spot_klines_data/BTCUSDT_15m_2024-2025.csv')
df_5m = pd.read_csv('spot_klines_data/BTCUSDT_5m_2024-2025.csv')
df_1m = pd.read_csv('spot_klines_data/BTCUSDT_1m_2024-2025.csv')

results = backtest_multi_timeframe(df_15m, df_5m, df_1m)
print(results)


2025-02-16 11:06:03.173 | INFO     | __main__:backtest_multi_timeframe:7 - Condition meet 15m
2025-02-16 11:06:03.240 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 0
2025-02-16 11:06:03.292 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 1
2025-02-16 11:06:03.340 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 2
2025-02-16 11:06:03.387 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 3
2025-02-16 11:06:03.434 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 4
2025-02-16 11:06:03.482 | INFO     | __main__:backtest_multi_timeframe:17 - Condition meet until 15m  and no opportunity in 5m for data point 5
2025-02-16 11:06:03.527 | INFO     | __mai

KeyboardInterrupt: 